### TODO
- add_road_info
  - 交通データとマスタデータで区間の持ち方が違うためマージ時にNaNが生まれる
  - デフォルトで80km/hを設定するようにしているが、最終的には区間の方を修正する必要がありそう

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [14]:
# data directory
PROCESSED_DATA_DIR = '../Input_processed_data'
ORI_DATA_DIR = '../Input_original_data'

# IC, 道路情報 csv
IC_SUB_CSV = f'{PROCESSED_DATA_DIR}/road_master/tateyama_kannetsu_ic.csv'
IC_NET_SUB_CSV = f'{PROCESSED_DATA_DIR}/road_master/tateyama_kannetsu_doronet_sub_with_rule.csv'

# 交通量 csv
TRAFFIC_TATEYAMA_CSV = f'{PROCESSED_DATA_DIR}/traffic/traffic_tateyama.csv'
TRAFFIC_KANNETSU_CSV = f'{PROCESSED_DATA_DIR}/traffic/traffic_kannetsu.csv'

# 天気 csv
WEATHER_CSV = f'{PROCESSED_DATA_DIR}/weather_data/weather_20220401.csv'

# 検索量 csv
SEARCH_COUNT_DIR = f'{PROCESSED_DATA_DIR}/search_count'

SEARCH_COUNT_TATEYAMA_CSV = f'{SEARCH_COUNT_DIR}/search-count_tateyama.csv'
SEARCH_COUNT_KANNETSU_CSV = f'{SEARCH_COUNT_DIR}/search-count_kannetsu.csv'
SEARCH_COUNT_CSV = lambda road: SEARCH_COUNT_TATEYAMA_CSV if road == 'tateyama' else SEARCH_COUNT_KANNETSU_CSV

SEARCH_COUNT_TATEYAMA_UNSPECIFIED_CSV = f'{SEARCH_COUNT_DIR}/search-count_tateyama_unspecified.csv'
SEARCH_COUNT_KANNETSU_UNSPECIFIED_CSV = f'{SEARCH_COUNT_DIR}/search-count_kannetsu_unspecified.csv'
SEARCH_COUNT_UNSPECIFIED_CSV = lambda road: SEARCH_COUNT_TATEYAMA_UNSPECIFIED_CSV if road == 'tateyama' else SEARCH_COUNT_KANNETSU_UNSPECIFIED_CSV

# 渋滞量 csv
JAM_CSV = f'{PROCESSED_DATA_DIR}/jam_data/jam_tateyama_kannetsu_2021.csv'

In [9]:
df_tateyama = pd.read_csv(TRAFFIC_TATEYAMA_CSV, index_col='datetime', parse_dates=True, 
                          dtype={'start_code': str, 'end_code': str}).reset_index()
df_kannetsu = pd.read_csv(TRAFFIC_KANNETSU_CSV, index_col='datetime', parse_dates=True, 
                          dtype={'start_code': str, 'end_code': str}).reset_index()

In [25]:
def add_ic_info(df):
  usecols = ['ic_code', 'area_code', 'pref_code', 'type', 'lat', 'lng', 'degree']
  df_ic = pd.read_csv(IC_SUB_CSV, usecols=usecols, dtype={'ic_code': str})

  df = pd.merge(df, df_ic, left_on='start_code', right_on='ic_code', how='left')
  df.drop('ic_code', axis=1, inplace=True)
  df.rename(columns=lambda c: 'start_'+c if c in usecols else c, inplace=True)

  df = pd.merge(df, df_ic.drop('pref_code', axis=1), left_on='end_code', right_on='ic_code', how='left')
  df.drop('ic_code', axis=1, inplace=True)
  df.rename(columns=lambda c: 'end_'+c if c in usecols else c, inplace=True)

  return df


def add_road_info(df):
  usecols = ['start_code', 'end_code', 'limit']
  df_icnet = pd.read_csv(IC_NET_SUB_CSV, usecols=usecols, dtype={'start_code': str, 'end_code': str})

  df = pd.merge(df, df_icnet, on=['start_code', 'end_code'], how='left')
  default_limit = 80
  df['limit'].fillna(default_limit, inplace=True)
  
  return df


def add_weather_info(df):
  colnames = ['date0', 'temperature', 'rainfall', 'day_length', 'snowfall', 'snow_depth', 'wind_speed', 'humidity', 'air_pressure', 'cloudiness', 'pref_code', 'pref_name', 'date']
  df_weather = pd.read_csv(WEATHER_CSV, index_col=0, header=0, names=colnames)

  df['date'] = df['datetime'].dt.date.astype(str)
  df = pd.merge(df, df_weather, left_on=['date', 'start_pref_code'], right_on=['date', 'pref_code'], how='left')
  df.drop(['date', 'date0', 'pref_name'], axis=1, inplace=True)

  return df


def add_search_count(df, df_count):
  df = pd.merge(df, df_count, how='left',
                left_on=['datetime', 'start_code', 'end_code'], 
                right_on=['passing_time', 'start_code', 'end_code'])
  df.drop('passing_time', axis=1, inplace=True)
  df['search'].fillna(0, inplace=True)

  return df


def add_unspecified_search_count(df, df_count_unspecified):
  df['date'] = df['datetime'].dt.date.astype(str)

  df_count_unspecified['search'] = df_count_unspecified.groupby(['start_code','end_code'])['search'].shift(1)
  df_count_unspecified.rename(columns={'search': 'unspecified_search'}, inplace=True)

  df = pd.merge(df, df_count_unspecified, how='left',
                left_on=['date', 'start_code', 'end_code'],
                right_on=['search_date', 'start_code', 'end_code'])
  df.drop(['date', 'search_date'], axis=1, inplace=True)
  df['unspecified_search'].fillna(0, inplace=True)
  
  return df


def add_jam_info(df):
  df_jam = pd.read_csv(JAM_CSV, index_col='datetime', parse_dates=True, 
                        dtype={'start_code': str, 'end_code': str, 'road_code': str}).reset_index()
  df = pd.merge(df, df_jam, how='left', on=['datetime', 'start_code', 'end_code'])
  df.drop(['road_code'], axis=1, inplace=True)
  df['jam_quantity'].fillna(0, inplace=True)

  return df

In [11]:
def preprocess(df, road):
  assert road in {'tateyama', 'kannetsu'}, 'road should be "tateyama" or "kannetsu"'
  
  # IC情報をマージ
  df = add_ic_info(df)
  # 区間情報をマージ（制限速度）
  df = add_road_info(df)
  # 天気情報をマージ
  df = add_weather_info(df)
  # 時間指定ありの検索をマージ
  df_count = pd.read_csv(SEARCH_COUNT_CSV(road), index_col='passing_time', parse_dates=True, 
                        dtype={'start_code': str, 'end_code': str}).reset_index()
  df = add_search_count(df, df_count)
  # 時間指定なしの検索をマージ
  df_count_unspecified = pd.read_csv(SEARCH_COUNT_UNSPECIFIED_CSV(road),
                                    dtype={'start_code': str, 'end_code': str})
  df = add_unspecified_search_count(df, df_count_unspecified)
  # 渋滞量をマージ
  df = add_jam_info(df)

  return df

In [16]:
tateyama = preprocess(df_tateyama, 'tateyama')
tateyama.head()

,datetime,is_holiday,start_name,end_name,start_code,end_code,direction,total,speed,KP,start_area_code,start_pref_code,start_type,start_lat,start_lng,start_degree,end_area_code,end_type,end_lat,end_lng,end_degree,temperature,rainfall,day_length,snowfall,snow_depth,wind_speed,humidity,air_pressure,cloudiness,pref_code,search,unspecified_search,jam_quantity
0,2021-04-01 00:00:00,平,市原,京葉道路・館山自動車道接続部,1130006,1130001,上り,8.0,92.0,39.17,3,12,IC,35.49471,140.094986,4.0,3,JCT,35.55136,140.143014,4.0,16.3,0.0,6.1,0.0,0.0,3.8,56.0,1020.1,NaN,12.0,0.0,0.0,0.0
1,2021-04-01 00:05:00,平,市原,京葉道路・館山自動車道接続部,1130006,1130001,上り,4.0,88.0,39.17,3,12,IC,35.49471,140.094986,4.0,3,JCT,35.55136,140.143014,4.0,16.3,0.0,6.1,0.0,0.0,3.8,56.0,1020.1,NaN,12.0,0.0,0.0,0.0
2,2021-04-01 00:10:00,平,市原,京葉道路・館山自動車道接続部,1130006,1130001,上り,9.0,83.0,39.17,3,12,IC,35.49471,140.094986,4.0,3,JCT,35.55136,140.143014,4.0,16.3,0.0,6.1,0.0,0.0,3.8,56.0,1020.1,NaN,12.0,0.0,0.0,0.0
3,2021-04-01 00:15:00,平,市原,京葉道路・館山自動車道接続部,1130006,1130001,上り,8.0,94.0,39.17,3,12,IC,35.49471,140.094986,4.0,3,JCT,35.55136,140.143014,4.0,16.3,0.0,6.1,0.0,0.0,3.8,56.0,1020.1,NaN,12.0,0.0,0.0,0.0
4,2021-04-01 00:20:00,平,市原,京葉道路・館山自動車道接続部,1130006,1130001,上り,5.0,115.0,39.17,3,12,IC,35.49471,140.094986,4.0,3,JCT,35.55136,140.143014,4.0,16.3,0.0,6.1,0.0,0.0,3.8,56.0,1020.1,NaN,12.0,0.0,0.0,0.0


In [17]:
tateyama.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1857024 entries, 0 to 1857023
Data columns (total 34 columns):
 #   Column              Dtype         
---  ------              -----         
 0   datetime            datetime64[ns]
 1   is_holiday          object        
 2   start_name          object        
 3   end_name            object        
 4   start_code          object        
 5   end_code            object        
 6   direction           object        
 7   total               float64       
 8   speed               float64       
 9   KP                  float64       
 10  start_area_code     int64         
 11  start_pref_code     int64         
 12  start_type          object        
 13  start_lat           float64       
 14  start_lng           float64       
 15  start_degree        float64       
 16  end_area_code       int64         
 17  end_type            object        
 18  end_lat             float64       
 19  end_lng             float64       
 20  en

In [19]:
kannetsu = preprocess(df_kannetsu, 'kannetsu')
kannetsu.head()

,datetime,is_holiday,start_name,end_name,start_code,end_code,direction,total,speed,KP,start_area_code,start_pref_code,start_type,start_lat,start_lng,start_degree,end_area_code,end_type,end_lat,end_lng,end_degree,temperature,rainfall,day_length,snowfall,snow_depth,wind_speed,humidity,air_pressure,cloudiness,pref_code,search,unspecified_search,jam_quantity
0,2021-04-01 00:00:00,平,所沢,大泉ＪＣＴ,1800006,1110210,上り,86.0,86.0,5.0,3,11,IC,35.80615,139.535511,4.0,3,JCT,35.75582,139.601514,8.0,16.3,0.0,7.4,NaN,NaN,2.7,NaN,NaN,NaN,11.0,0.0,0.0,0.0
1,2021-04-01 00:05:00,平,所沢,大泉ＪＣＴ,1800006,1110210,上り,64.0,91.0,5.0,3,11,IC,35.80615,139.535511,4.0,3,JCT,35.75582,139.601514,8.0,16.3,0.0,7.4,NaN,NaN,2.7,NaN,NaN,NaN,11.0,0.0,0.0,0.0
2,2021-04-01 00:10:00,平,所沢,大泉ＪＣＴ,1800006,1110210,上り,64.0,91.0,5.0,3,11,IC,35.80615,139.535511,4.0,3,JCT,35.75582,139.601514,8.0,16.3,0.0,7.4,NaN,NaN,2.7,NaN,NaN,NaN,11.0,0.0,0.0,0.0
3,2021-04-01 00:15:00,平,所沢,大泉ＪＣＴ,1800006,1110210,上り,55.0,87.0,5.0,3,11,IC,35.80615,139.535511,4.0,3,JCT,35.75582,139.601514,8.0,16.3,0.0,7.4,NaN,NaN,2.7,NaN,NaN,NaN,11.0,0.0,0.0,0.0
4,2021-04-01 00:20:00,平,所沢,大泉ＪＣＴ,1800006,1110210,上り,52.0,87.0,5.0,3,11,IC,35.80615,139.535511,4.0,3,JCT,35.75582,139.601514,8.0,16.3,0.0,7.4,NaN,NaN,2.7,NaN,NaN,NaN,11.0,0.0,0.0,0.0


In [20]:
kannetsu.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7312032 entries, 0 to 7312031
Data columns (total 34 columns):
 #   Column              Dtype         
---  ------              -----         
 0   datetime            datetime64[ns]
 1   is_holiday          object        
 2   start_name          object        
 3   end_name            object        
 4   start_code          object        
 5   end_code            object        
 6   direction           object        
 7   total               float64       
 8   speed               float64       
 9   KP                  float64       
 10  start_area_code     int64         
 11  start_pref_code     int64         
 12  start_type          object        
 13  start_lat           float64       
 14  start_lng           float64       
 15  start_degree        float64       
 16  end_area_code       int64         
 17  end_type            object        
 18  end_lat             float64       
 19  end_lng             float64       
 20  en